### Mantle rheology

Adjust mantle rheology from aspect

### read rheology from updated rheology

For reference, look at explain_update_modHK03_rheology.pdf


In [ ]:
# 

HK03 = /
{/
    "dislocation_creep":/
    {/
        "A": 90,/
        "p": 0.0,/
        "r": 1.2,/
        "n": 3.5,/
        "E": 480e3,/
        "V": 11e-6,/
        "d" : 1e4,/
        "Coh" : 1000.0/
    },/
    "diffusion_creep":/
    {/
        "A" : 1.0e6,/
        "p" : 3.0,/
        "r" : 1.0,/
        "n" : 1.0,/
        "E" : 335e3,/
        "V" : 4e-6,/
        "d" : 1e4,/
        "Coh" : 1000.0/
    }/
}

### Read aspect depth_average file

I would be called later with the class function, so the class could know about the temperature and pressure profile

In [ ]:
def ReadAspectProfile(depth_average_path):
    """
    read a T,P profile from aspect's depth average file
    """
    # check file exist
    assert(os.access(depth_average_path, os.R_OK))
    # read that
    DepthAverage = DEPTH_AVERAGE_PLOT('DepthAverage')
    DepthAverage.ReadHeader(depth_average_path)
    DepthAverage.ReadData(depth_average_path)
    DepthAverage.SplitTimeStep()
    time_step = 0
    i0 = DepthAverage.time_step_indexes[time_step][-1] * DepthAverage.time_step_length
    if time_step == len(DepthAverage.time_step_times) - 1:
        # this is the last step
        i1 = DepthAverage.data.shape[0]
    else:
        i1 = DepthAverage.time_step_indexes[time_step + 1][0] * DepthAverage.time_step_length
    data = DepthAverage.data[i0:i1, :]
    col_depth = DepthAverage.header['depth']['col']
    col_P = DepthAverage.header['adiabatic_pressure']['col']
    col_T = DepthAverage.header['temperature']['col']
    depths = data[:, col_depth]
    pressures = data[:, col_P]
    temperatures = data[:, col_T]
    return depths, pressures, temperatures


### Class of rheology

Here is a class for rheologies. For the purpose of illustrating the modified HK03 rheology, I only include this.

#### MantleRheology_v0 function

* Take a reference rheology(i.e. diffusion creep and dislocation creep) and apply a perturbation in E and V.
* Compute the upper mantle rheology with the reference profile
* Compute the lower mantle rheology(A, E) with a jump on 660 km(30.0 in default)
* Plot whole mantle rheology profile in semilog plot and output a json file containing the values of parameters in the rheology

### initial a class object and call member functions

Here I initiate a class and call the member functions to plot the rhoelogy profile.

In [ ]:
Operator = RHEOLOGY_OPR()
# read profile
Operator.ReadProfile(file_path)
# do a random walk
save_profile = 1
include_lower_mantle = 1
version = 1
if version == 0:
    Operator.MantleRheology_v0(save_profile=save_profile)
else:
    raise CheckValueError('%d is not a valid version of Mantle Rheology' % version)